In [1]:
import requests
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from dotenv import load_dotenv

In [2]:
 # Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv('NOAA_CLIMATE_DATA')
token = api_key

In [3]:
def get_weather_data(token, location_id, start_date, end_date, data_type):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    headers = {
        'token': token
    }
    params = {
        'datasetid': 'GSOY',  # Global Historical Climatology Network - GSOM is monthly, GSOY is yearly
        'locationid': f'FIPS:{location_id}',  # Location ID for country
        'startdate': start_date,
        'enddate': end_date,
        'datatypeid': data_type,
        'units': 'standard',  # Use 'standard' for Fahrenheit, inches, etc.
        'limit': 1000  # Adjust as needed
    }

    response = requests.get(url, headers=headers, params=params)
    print(response)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.status_code, response.text



In [4]:
# list of dates
start_dates = [
    '1995-01-01', '2000-01-01', '2005-01-01', '2010-01-01',
    '2015-01-01', '2020-01-01', '2023-01-01'
]



end_dates = [  
    '1999-12-31', '2004-12-31', '2009-12-31', '2014-12-31',
    '2019-12-31', '2023-12-31'
]

# Date types to get
data_types = ["DP10", "DP1X", "DT32", "DX70", "DX90", "PRCP", "RHAV", "TAVG", 'TMAX']

In [5]:
# Load country codes to data frame
codes_path = 'Resources/ghcnd_country_codes.csv'
code_country_df = pd.read_csv(codes_path, sep=',', header=0)
location_ids = code_country_df['Code'].tolist()
print(code_country_df.head())

  Code
0   AE
1   AF
2   AG
3   AJ
4   AL


In [6]:
# Loop through each country and date range
for location_id in location_ids:
    for start_date, end_date in zip(start_dates, end_dates):

        for data_type in data_types:
            weather_data = get_weather_data(token, location_id, start_date, end_date, data_type)
            
            # Check if the response is successful or not
            if isinstance(weather_data, tuple):
                # Error case
                print("Error occurred: Status Code:", weather_data[0], "Message:", weather_data[1])
        else:
            # Success case
            # Check if the response contains any data
            if weather_data.get("results"):
                # Parse the JSON strings into dictionaries
                results = weather_data["results"]

                # Create an empty DataFrame
                columns = ["station", "date"] + data_types
                df = pd.DataFrame(columns=columns)

                # Iterate over each result
                for result in results:
                    date = result["date"][:10]  # Extract date portion
                    datatype = result["datatype"]
                    value = result["value"]
                    station = result["station"]
        
                    # Check if the date already exists in the DataFrame
                    if date not in df.index:
                        df.loc[date, "station"] = station
        
                    # Add the value to the corresponding cell
                    df.loc[date, datatype] = value

                # Reset index to make date a column
                df.reset_index(inplace=True)
                # Rename the 'index' column to 'date'
                df.rename(columns={"index": "date"}, inplace=True)

                print(df)
    
            # Constructing filename
            filename = f"ClimateData/{location_id}_{start_date}_{end_date}_weather_data.csv"
        
            # Export the final dataframe to a CSV file with the constructed filename
            df.to_csv(filename, index=False)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [503]>
Error occurred: Status Code: 503 Message: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>503 Service Unavailable</title>
</head><body>
<h1>Service Unavailable</h1>
<p>The server is temporarily unable to service your
request due to maintenance downtime or capacity
problems. Please try again later.</p>
<p>Additionally, a 503 Service Unavailable
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>



AttributeError: 'tuple' object has no attribute 'get'